In [3]:
# =========================================
# Spotify Churn EDA (Origin vs Generated)
# - Fig01 ~ Fig10 자동 저장
# - matplotlib only (no seaborn)
# =========================================

import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -------------------------
# 0) 경로 설정 (필요 시 수정)
# -------------------------
ORIGIN_PATH = "data/spotify_churn_dataset_origin.csv"
GEN_PATH    = "data/spotify_churn_dataset copy.csv"

OUT_DIR = Path("images")
OUT_DIR.mkdir(parents=True, exist_ok=True)

TARGET_COL = "is_churned"

KR = {
    "is_churned": "이탈 여부(1=이탈, 0=유지)",
    "subscription_type": "구독 유형",
    "offline_listening": "오프라인 사용(1=사용, 0=미사용)",
    "ads_listened_per_week": "주간 광고 노출 수",
    "listening_time": "청취 시간(분)",
    "songs_played_per_day": "일일 재생 곡 수",
    "skip_rate": "스킵 비율",
    "country": "국가",
    "device_type": "사용 기기",
    "gender": "성별",
    "age": "나이",
}

# -----------------------------------
# 1) 한글 폰트 설정 (윈도우/맥/리눅스)
# -----------------------------------
def set_korean_font():
    import matplotlib
    from matplotlib import font_manager

    candidates = [
        "Malgun Gothic",      # Windows
        "AppleGothic",        # macOS
        "NanumGothic",        # common on Linux
        "Noto Sans CJK KR",   # common
        "Noto Sans KR",
    ]

    available = {f.name for f in font_manager.fontManager.ttflist}
    for name in candidates:
        if name in available:
            matplotlib.rcParams["font.family"] = name
            break

    # 마이너스 기호 깨짐 방지
    matplotlib.rcParams["axes.unicode_minus"] = False

set_korean_font()

# -------------------------
# 2) 공통 유틸
# -------------------------
def savefig(fig, filename: str):
    path = OUT_DIR / filename
    fig.savefig(path, dpi=200, bbox_inches="tight")
    print(f"[Saved] {path}")

def pct(x):
    return f"{x*100:.2f}%"

def add_bar_labels(ax, bars, labels, dy=0.01, fontsize=9):
    """bars 위에 labels 텍스트를 붙임"""
    ymax = ax.get_ylim()[1]
    for b, lab in zip(bars, labels):
        if b is None:
            continue
        h = b.get_height()
        ax.text(
            b.get_x() + b.get_width()/2,
            h + ymax*dy,
            lab,
            ha="center",
            va="bottom",
            fontsize=fontsize,
        )

def style_ax(ax, title, xlabel=None, ylabel=None):
    ax.set_title(title, pad=10)
    if xlabel: ax.set_xlabel(xlabel)
    if ylabel: ax.set_ylabel(ylabel)
    ax.grid(True, axis="y", alpha=0.3)

# -------------------------
# 3) 데이터 로드
# -------------------------
origin = pd.read_csv(ORIGIN_PATH)
gen    = pd.read_csv(GEN_PATH)

# -------------------------
# 4) Fig01: 타깃 분포 비교
# -------------------------
def fig01_target_dist_compare(df_a, df_b, label_a="Origin", label_b="Generated"):
    # 0/1 counts
    def counts(df):
        c = df[TARGET_COL].value_counts().reindex([0,1], fill_value=0)
        total = len(df)
        return c, total

    c_a, n_a = counts(df_a)
    c_b, n_b = counts(df_b)

    cats = ["유지(0)", "이탈(1)"]
    x = np.arange(len(cats))
    w = 0.35

    fig, ax = plt.subplots(figsize=(7,4))
    bars_a = ax.bar(x - w/2, [c_a[0], c_a[1]], width=w, label=label_a)
    bars_b = ax.bar(x + w/2, [c_b[0], c_b[1]], width=w, label=label_b)

    # 라벨: count + pct
    labs_a = [f"{c_a[0]:,}\n({c_a[0]/n_a:.1%})", f"{c_a[1]:,}\n({c_a[1]/n_a:.1%})"]
    labs_b = [f"{c_b[0]:,}\n({c_b[0]/n_b:.1%})", f"{c_b[1]:,}\n({c_b[1]/n_b:.1%})"]
    add_bar_labels(ax, bars_a, labs_a, dy=0.015)
    add_bar_labels(ax, bars_b, labs_b, dy=0.015)

    ax.set_xticks(x)
    ax.set_xticklabels(cats)
    style_ax(ax, "타깃 분포 비교(Origin vs Generated)", ylabel="사용자 수")
    ax.legend()

    savefig(fig, "fig01_target_dist_compare.png")
    plt.close(fig)

fig01_target_dist_compare(origin, gen)

# -------------------------
# 5) Fig02/03: 히스토그램 + 음수값 체크
# -------------------------
def hist_with_negative_check(df, col, title, filename, bins=40):
    neg_cnt = int((df[col] < 0).sum())
    fig, ax = plt.subplots(figsize=(7,4))
    ax.hist(df[col].values, bins=bins)
    ax.axvline(0, linestyle="--", linewidth=1)

    style_ax(ax, title, xlabel=KR.get(col,col), ylabel="빈도")
    ax.text(
        0.99, 0.95,
        f"음수 값 개수: {neg_cnt:,}",
        transform=ax.transAxes,
        ha="right", va="top", fontsize=10
    )
    savefig(fig, filename)
    plt.close(fig)

hist_with_negative_check(gen, "listening_time",
                        "Generated - 청취 시간 분포(음수값 존재 여부)",
                        "fig02_listening_time_hist.png")

hist_with_negative_check(gen, "songs_played_per_day",
                        "Generated - 일일 재생 곡 수 분포(음수값 존재 여부)",
                        "fig03_songs_hist.png")

# -------------------------
# 6) 카테고리별 이탈률 비교(Origin vs Generated)
# -------------------------
def churn_rate_by_category(df, col):
    g = df.groupby(col)[TARGET_COL].agg(["mean","count"]).reset_index()
    g.rename(columns={"mean":"churn_rate","count":"n"}, inplace=True)
    return g

def fig_rate_compare_cat(df_a, df_b, col, title, filename, label_a="Origin", label_b="Generated"):
    a = churn_rate_by_category(df_a, col)
    b = churn_rate_by_category(df_b, col)

    # 카테고리 합집합 기준 정렬
    cats = sorted(set(a[col].astype(str)) | set(b[col].astype(str)))

    a_map = {str(r[col]): (r["churn_rate"], r["n"]) for _, r in a.iterrows()}
    b_map = {str(r[col]): (r["churn_rate"], r["n"]) for _, r in b.iterrows()}

    a_rates = [a_map.get(c, (np.nan, 0))[0] for c in cats]
    b_rates = [b_map.get(c, (np.nan, 0))[0] for c in cats]
    a_ns    = [a_map.get(c, (np.nan, 0))[1] for c in cats]
    b_ns    = [b_map.get(c, (np.nan, 0))[1] for c in cats]

    x = np.arange(len(cats))
    w = 0.35

    fig, ax = plt.subplots(figsize=(10,4))
    bars_a = ax.bar(x - w/2, np.nan_to_num(a_rates, nan=0.0), width=w, label=label_a)
    bars_b = ax.bar(x + w/2, np.nan_to_num(b_rates, nan=0.0), width=w, label=label_b)

    # missing은 텍스트로 표시
    for i, c in enumerate(cats):
        if np.isnan(a_rates[i]):
            ax.text(i - w/2, 0.01, "N/A", ha="center", va="bottom", fontsize=9)
        if np.isnan(b_rates[i]):
            ax.text(i + w/2, 0.01, "N/A", ha="center", va="bottom", fontsize=9)

    labs_a = [("N/A" if np.isnan(r) else f"{r*100:.1f}%\n(n={n:,})") for r, n in zip(a_rates, a_ns)]
    labs_b = [("N/A" if np.isnan(r) else f"{r*100:.1f}%\n(n={n:,})") for r, n in zip(b_rates, b_ns)]
    add_bar_labels(ax, bars_a, labs_a, dy=0.02, fontsize=8)
    add_bar_labels(ax, bars_b, labs_b, dy=0.02, fontsize=8)

    ax.set_xticks(x)
    ax.set_xticklabels(cats)
    ax.set_ylim(0, max([v for v in a_rates+b_rates if not np.isnan(v)] + [0.1]) * 1.25)

    style_ax(ax, title, xlabel=KR.get(col,col), ylabel="이탈률")
    ax.legend()

    savefig(fig, filename)
    plt.close(fig)

# Fig04: subscription_type
fig_rate_compare_cat(
    origin, gen,
    "subscription_type",
    "구독 유형별 이탈률 비교(Origin vs Generated)",
    "fig04_subscription_churn_compare.png"
)

# Fig05: offline_listening
# (0/1을 보기 좋게 문자열로 바꾸어 비교)
origin_off = origin.copy()
gen_off    = gen.copy()
origin_off["offline_listening"] = origin_off["offline_listening"].map({0:"미사용(0)", 1:"사용(1)"})
gen_off["offline_listening"]    = gen_off["offline_listening"].map({0:"미사용(0)", 1:"사용(1)"})

fig_rate_compare_cat(
    origin_off, gen_off,
    "offline_listening",
    "오프라인 사용 여부별 이탈률 비교(Origin vs Generated)",
    "fig05_offline_churn_compare.png"
)

# -------------------------
# 7) Fig06: 광고 분포
# -------------------------
def fig06_ads_dist(df):
    fig, ax = plt.subplots(figsize=(7,4))
    ax.hist(df["ads_listened_per_week"].values, bins=30)
    style_ax(ax, "Generated - 주간 광고 노출 수 분포", xlabel=KR["ads_listened_per_week"], ylabel="빈도")
    savefig(fig, "fig06_ads_dist.png")
    plt.close(fig)

fig06_ads_dist(gen)

# -------------------------
# 8) Fig07: 광고 값별 이탈률(라인)
# -------------------------
def fig07_ads_churn_by_value(df):
    g = df.groupby("ads_listened_per_week")[TARGET_COL].agg(["mean","count"]).reset_index()
    g = g.sort_values("ads_listened_per_week")

    fig, ax = plt.subplots(figsize=(9,4))
    ax.plot(g["ads_listened_per_week"], g["mean"], marker="o", linewidth=1)

    style_ax(ax, "Generated - 광고 노출 수(값)별 이탈률", xlabel=KR["ads_listened_per_week"], ylabel="이탈률")
    ax.set_ylim(0, min(1.0, g["mean"].max()*1.2))

    # 일부 포인트에 n 표시(너무 빽빽하면 생략)
    step = max(1, len(g)//15)
    for i in range(0, len(g), step):
        ax.text(g.iloc[i]["ads_listened_per_week"], g.iloc[i]["mean"], f"n={int(g.iloc[i]['count'])}", fontsize=8)

    savefig(fig, "fig07_ads_churn_by_value.png")
    plt.close(fig)

fig07_ads_churn_by_value(gen)

# -------------------------
# 9) Fig08: ads 임계점(<=15 vs >=16) Origin vs Generated
# -------------------------
def fig08_ads_threshold_compare(df_a, df_b, thr=15, label_a="Origin", label_b="Generated"):
    def stats(df):
        low  = df[df["ads_listened_per_week"] <= thr]
        high = df[df["ads_listened_per_week"] >  thr]
        return {
            f"≤{thr}": (low[TARGET_COL].mean(), len(low)),
            f"≥{thr+1}": (high[TARGET_COL].mean(), len(high))
        }

    a = stats(df_a)
    b = stats(df_b)

    cats = list(a.keys())  # ["≤15","≥16"]
    x = np.arange(len(cats))
    w = 0.35

    a_rates = [a[c][0] for c in cats]
    b_rates = [b[c][0] for c in cats]
    a_ns    = [a[c][1] for c in cats]
    b_ns    = [b[c][1] for c in cats]

    fig, ax = plt.subplots(figsize=(7,4))
    bars_a = ax.bar(x - w/2, a_rates, width=w, label=label_a)
    bars_b = ax.bar(x + w/2, b_rates, width=w, label=label_b)

    labs_a = [f"{r*100:.1f}%\n(n={n:,})" for r, n in zip(a_rates, a_ns)]
    labs_b = [f"{r*100:.1f}%\n(n={n:,})" for r, n in zip(b_rates, b_ns)]
    add_bar_labels(ax, bars_a, labs_a, dy=0.02, fontsize=9)
    add_bar_labels(ax, bars_b, labs_b, dy=0.02, fontsize=9)

    ax.set_xticks(x)
    ax.set_xticklabels(cats)
    style_ax(ax, f"광고 임계점 비교(≤{thr} vs ≥{thr+1})", ylabel="이탈률")
    ax.set_ylim(0, max(a_rates + b_rates) * 1.25)
    ax.legend()

    savefig(fig, "fig08_ads_threshold_compare.png")
    plt.close(fig)

fig08_ads_threshold_compare(origin, gen, thr=15)

# -------------------------
# 10) Fig09: subscription × offline churn heatmap (Generated)
# -------------------------
def fig09_sub_offline_heatmap(df):
    # (구독유형 x offline) 이탈률 + n
    pivot_rate = df.pivot_table(
        index="subscription_type",
        columns="offline_listening",
        values=TARGET_COL,
        aggfunc="mean"
    )
    pivot_n = df.pivot_table(
        index="subscription_type",
        columns="offline_listening",
        values=TARGET_COL,
        aggfunc="count"
    )

    # 컬럼 이름 보기 좋게
    col_map = {0:"미사용(0)", 1:"사용(1)"}
    pivot_rate = pivot_rate.rename(columns=col_map)
    pivot_n    = pivot_n.rename(columns=col_map)

    # 행 순서(원하면 바꿔도 됨)
    row_order = [c for c in ["Free","Student","Premium","Family"] if c in pivot_rate.index]
    pivot_rate = pivot_rate.loc[row_order]
    pivot_n    = pivot_n.loc[row_order]

    data = pivot_rate.values

    fig, ax = plt.subplots(figsize=(7,4))
    im = ax.imshow(data, aspect="auto")

    ax.set_xticks(np.arange(pivot_rate.shape[1]))
    ax.set_yticks(np.arange(pivot_rate.shape[0]))
    ax.set_xticklabels(pivot_rate.columns.tolist())
    ax.set_yticklabels(pivot_rate.index.tolist())

    ax.set_title("Generated - 구독 유형 × 오프라인 사용 이탈률(Heatmap)", pad=10)

    # 셀 수치 주석(이탈률 + n)
    for i in range(pivot_rate.shape[0]):
        for j in range(pivot_rate.shape[1]):
            r = pivot_rate.iloc[i, j]
            n = int(pivot_n.iloc[i, j])
            ax.text(j, i, f"{r*100:.1f}%\n(n={n:,})", ha="center", va="center", fontsize=9)

    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04, label="이탈률")
    savefig(fig, "fig09_sub_offline_heatmap.png")
    plt.close(fig)

fig09_sub_offline_heatmap(gen)

# -------------------------
# 11) Fig10: 세그먼트 요약(표 이미지 + CSV 저장)
# -------------------------
def fig10_segment_summary(df, thr=15):
    segs = [
        {
            "segment": "고위험: Free & ads≥16 & offline=0",
            "mask": (df["subscription_type"]=="Free") & (df["ads_listened_per_week"]>thr) & (df["offline_listening"]==0),
        },
        {
            "segment": "저위험: Free & ads≤15 & offline=1",
            "mask": (df["subscription_type"]=="Free") & (df["ads_listened_per_week"]<=thr) & (df["offline_listening"]==1),
        }
    ]

    rows = []
    N = len(df)
    for s in segs:
        sub = df[s["mask"]]
        rows.append({
            "Segment": s["segment"],
            "Users(n)": len(sub),
            "Share": len(sub)/N,
            "ChurnRate": sub[TARGET_COL].mean() if len(sub)>0 else np.nan
        })

    seg_df = pd.DataFrame(rows)
    seg_df["Share"] = seg_df["Share"].map(lambda x: f"{x*100:.2f}%")
    seg_df["ChurnRate"] = seg_df["ChurnRate"].map(lambda x: f"{x*100:.2f}%")

    # CSV 저장
    seg_csv = OUT_DIR / "fig10_segment_summary_table.csv"
    seg_df.to_csv(seg_csv, index=False, encoding="utf-8-sig")
    print(f"[Saved] {seg_csv}")

    # 표 이미지 저장
    fig, ax = plt.subplots(figsize=(10,2.2))
    ax.axis("off")
    table = ax.table(
        cellText=seg_df.values,
        colLabels=seg_df.columns,
        cellLoc="center",
        loc="center",
    )
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 1.5)
    ax.set_title("Generated - 핵심 세그먼트 요약", pad=10)

    savefig(fig, "fig10_segment_summary_table.png")
    plt.close(fig)

fig10_segment_summary(gen, thr=15)

print("\n✅ Done. images/ 폴더에 Fig01~Fig10 저장 완료")


[Saved] images\fig01_target_dist_compare.png
[Saved] images\fig02_listening_time_hist.png
[Saved] images\fig03_songs_hist.png
[Saved] images\fig04_subscription_churn_compare.png
[Saved] images\fig05_offline_churn_compare.png
[Saved] images\fig06_ads_dist.png
[Saved] images\fig07_ads_churn_by_value.png
[Saved] images\fig08_ads_threshold_compare.png
[Saved] images\fig09_sub_offline_heatmap.png
[Saved] images\fig10_segment_summary_table.csv
[Saved] images\fig10_segment_summary_table.png

✅ Done. images/ 폴더에 Fig01~Fig10 저장 완료
